In [1]:
def read_from_file_with_bwt(filename):
    string = ''
    with open(filename, "r", encoding='utf-8') as file:
        i = 1
        tmp_string = ''
        for line in file:
            tmp_string += line
            if i % 300 == 0:
                suffix_array = naive_suffix_array(tmp_string)
                string += bwt_last_column(tmp_string, suffix_array) + '$'
                i = 1
                tmp_string = ''
            i += 1
        suffix_array = naive_suffix_array(tmp_string)
        string += bwt_last_column(tmp_string, suffix_array) + '$'
    return string

In [2]:
def naive_suffix_array(text):
    suffixes = [(text[i:], i) for i in range(len(text))]
    suffixes.sort(key=lambda x: x[0])
    return [suffix[1] for suffix in suffixes]

In [3]:
def bwt_last_column(text, suffix_array):
    bwt = ''.join(text[i - 1] for i in suffix_array)
    return ''.join(bwt)

In [4]:
# Python program for the above approach

import string

# Structure to store info of a node of linked list
class Node:
	def __init__(self, data):
		self.data = data
		self.next = None

# Does insertion at end in the linked list
def addAtLast(head, nn):
	if head is None:
		head = nn
		return head
	temp = head
	while temp.next is not None:
		temp = temp.next
	temp.next = nn
	return head

# Computes l_shift[]
def computeLShift(head, index, l_shift):
	l_shift[index] = head.data
	head = head.next

# Compares the characters of bwt_arr[] and sorts them alphabetically
def cmpfunc(a, b):
	return ord(a) - ord(b)

def invert(bwt_arr):
	len_bwt = len(bwt_arr)
	sorted_bwt = sorted(bwt_arr)
	l_shift = [0] * len_bwt

	# Index at which original string appears
	# in the sorted rotations list
	x = 4

	# Array of lists to compute l_shift
	arr = [[] for i in range(256)]

	# Adds each character of bwt_arr to a linked list
	# and appends to it the new node whose data part
	# contains index at which character occurs in bwt_arr
	for i in range(len_bwt):
		arr[ord(bwt_arr[i])].append(i)

	# Adds each character of sorted_bwt to a linked list
	# and finds l_shift
	for i in range(len_bwt):
		l_shift[i] = arr[ord(sorted_bwt[i])].pop(0)

	# Decodes the bwt
	decoded = [''] * len_bwt
	for i in range(len_bwt):
		x = l_shift[x]
		decoded[len_bwt-1-i] = bwt_arr[x]
	decoded_str = ''.join(decoded)

	return decoded_str[::-1]

# Driver program to test functions above
if __name__ == "__main__":
	bwt_arr = "annb$aa"
	invert(bwt_arr)

In [5]:
def bwt_bloks_decoding(string):
    tmp_string = ''
    decoded_string = ''
    for sub in string:
        if sub != '$':
            tmp_string += sub
        else:
            inverted_string = invert(tmp_string)
            if inverted_string != None:
                decoded_string += invert(tmp_string)
            tmp_string = ''
    return decoded_string

In [6]:
def move_to_front_encode(s):
    alphabet = list(range(256))  # Создаем алфавит из всех возможных символов (ASCII)
    encoded = []

    for char in s:
        index = alphabet.index(ord(char))  # Находим индекс символа в алфавите
        encoded.append(index)  # Добавляем индекс в закодированную строку
        # Перемещаем символ в начало алфавита
        alphabet.insert(0, alphabet.pop(index))

    return encoded

def move_to_front_decode(encoded):
    alphabet = list(range(256))  # Создаем алфавит из всех возможных символов (ASCII)
    decoded = []

    for index in encoded:
        char = chr(alphabet[index])  # Находим символ по индексу в алфавите
        decoded.append(char)  # Добавляем символ в декодированную строку
        # Перемещаем символ в начало алфавита
        alphabet.insert(0, alphabet.pop(index))

    return ''.join(decoded)

In [7]:
def arithmetic_coding_encode(input_string, frequency_dict, step=4):
    # Инициализируем начальные значения
    low = 0.0
    high = 1.0
    values_list = []

    i = 0
    # Проходимся по каждому символу в строке
    for symbol in input_string:

        # Обновляем границы интервала
        symbol_index = list(frequency_dict.keys()).index(symbol)
        new_low = low + (high - low) * (list(frequency_dict.values())[symbol_index - 1] if symbol_index != 0 else 0)
        new_high = low + (high - low) * list(frequency_dict.values())[symbol_index]

        # Обновляем нижнюю границу
        low = new_low

        # Обновляем верхнюю границу
        high = new_high

        i += 1
        if i == step or step * len(values_list) + i == len(input_string):
            values_list.append((low + high) / 2)
            low = 0.0
            high = 1.0
            i = 0

    return values_list

In [8]:
def arithmetic_coding_decode(encoded_values, frequency_dict, input_string_length, step=4):
    # Восстанавливаем исходную строку
    decoded_string = ""

    i = 0
    k = 0
    for _ in range(input_string_length):
        # Ищем символ в частотном словаре
        for symbol, symbol_high in frequency_dict.items():

            symbol_index = list(frequency_dict.keys()).index(symbol)
            symbol_low = list(frequency_dict.values())[symbol_index - 1] if symbol_index != 0 else 0
            # Если значение в пределах интервала для текущего символа
            if symbol_low <= encoded_values[k]+pow(10, -10) < symbol_high:
                decoded_string += symbol

                i += 1
                if i == step:
                    k += 1
                    i = 0
                else:
                    encoded_values[k] = (encoded_values[k] - symbol_low) / (symbol_high - symbol_low)

                break

    return decoded_string

In [9]:
from collections import Counter

def build_frequency_dict(text):
    frequency_dict = Counter(text)

    num_frequency_dict = {}
    point = 1 / sum(frequency_dict.values())
    num = 0
    for key in frequency_dict:
        num += point * frequency_dict[key]
        num_frequency_dict[key] = num

    return num_frequency_dict

In [10]:
import struct

def arithmetic_to_file(filename, encoded_text, dictionary, text_len):
    with open(filename, 'wb') as file:
        file.write(text_len.to_bytes(6, byteorder='big'))
        for item in encoded_text:
            file.write(bytearray(struct.pack("f", item)))
        for key in dictionary:
            file.write(ord(key).to_bytes(2, byteorder='big'))
            file.write(bytearray(struct.pack("f", dictionary[key])))
    return

In [11]:
import math

def arithmetic_from_file(filename, step=4):
    with open(filename, 'rb') as file:
        text_len = int.from_bytes(file.read(6), byteorder='big')
        encoded_text = []
        for i in range(math.ceil(text_len / step)):
            encoded_text.append(struct.unpack("f", file.read(4))[0])
        
        dictionary = {}
        while True:
            tmp = file.read(2)
            if not tmp:
                break
            letter = chr(int.from_bytes(tmp, byteorder='big'))
            count = struct.unpack("f", file.read(4))[0]
            dictionary[letter] = count
    
    return encoded_text, dictionary, text_len

In [12]:
def rle_encode(data):
    if type(data) == str: data += '$'
    else: data.append(0)
    encoded_data = []
    count = 1
    for i in range(1, len(data)):
        if data[i] == data[i-1]:
            count += 1
        else:
            if count > 1: encoded_data.append((data[i-1], count))
            else: encoded_data.append(data[i-1])
            count = 1
    index = -1
    labled_encoded_string = ''
    tmp_string = ''
    for i in range(len(encoded_data)):
        if not isinstance(encoded_data[i], tuple):
            tmp_string += encoded_data[i]
            if i == len(encoded_data)-1:
                if len(tmp_string) != 0:
                    labled_encoded_string += str(-len(tmp_string)) + tmp_string
                break
        else:
            if len(tmp_string) != 0:
                labled_encoded_string += str(-len(tmp_string)) + tmp_string
            labled_encoded_string += str(encoded_data[i][1]) + encoded_data[i][0]
            tmp_string = ''
    
    return labled_encoded_string

In [13]:
def rle_decode(filename, symbol_bytes, text_or_image):
    decoded_string = ''
    with open(filename, 'rb') as file:
        while True:
            read_bytes = file.read(2)
            if not read_bytes:
                break
            int_val = int.from_bytes(read_bytes, "big", signed=True)
            if int_val > 0:
                letter = chr(int.from_bytes(file.read(symbol_bytes), "big", signed=False))
                for _ in range(int_val):
                    decoded_string += letter
            else:
                for i in range(-int_val):
                    decoded_string += chr(int.from_bytes(file.read(symbol_bytes), "big", signed=False))
    return decoded_string

In [14]:
string = read_from_file_with_bwt('harry_potter_and_the_prisoner_of_azkaban_and_harry_potter_and_the_sorcerers_stone.txt')

In [15]:
MTF_encoded_string = move_to_front_encode(string)

In [16]:
string = ''
for item in MTF_encoded_string:
    string += chr(item)

In [17]:
RLE_encoded_string = rle_encode(string)

In [18]:
dictionary = build_frequency_dict(RLE_encoded_string)
encoded_text = arithmetic_coding_encode(RLE_encoded_string, dictionary)

In [19]:
arithmetic_to_file('encoded_harry_potter_and_the_prisoner_of_azkaban_and_harry_potter_and_the_sorcerers_stone.txt', encoded_text, dictionary, len(string))

In [25]:
encoded_text, dictionary, text_len = arithmetic_from_file('encoded_harry_potter_and_the_prisoner_of_azkaban_and_harry_potter_and_the_sorcerers_stone.txt')

In [27]:
AC_decoded_text = arithmetic_coding_decode(encoded_text, dictionary, text_len)

In [29]:
MTF_array = []
for sub in AC_decoded_text:
    MTF_array.append(ord(sub))

In [30]:
MTF_decoded_text = move_to_front_decode(MTF_array)

In [31]:
result = bwt_bloks_decoding(MTF_decoded_text)

IndexError: list index out of range

In [155]:
result

'\n  \n  \n   To Jill Prewett and Aine Kiely,\n   The godmothers of Swing\n  \n  \n   \n    1. OWL POST\n   \n   Harry Potter was a highly unusual boy in many ways. For one thing, he hated the summer holidays more than any other time of year. For another, he really wanted to do his homework but was forced to do it in secret, in the dead of night. And he also happened to be a wizard.\n   It was nearly midnight, and he was lying on his stomach in bed, the blankets drawn right over his head like a tent, a flashlight in one hand and a large leather bound book (A History of Magic by Bathilda Bagshot) propped open against the pillow. Harry moved the tip of his eagle feather quill down the page, frowning as he looked for something that would help him write his essay, "Witch Burning in the Fourteenth Century Was Completely Pointless Discuss."\n   The quill paused at the top of a likely looking paragraph. Harry pushed his round glasses up the bridge of his nose, moved his flashlight closer to t

In [156]:
import _pickle as pickle

In [157]:
output = open('tree.pkl', 'wb')
pickle.dump(tree, output, 2)
output.close()